# 18 Context Managers

Comprehensive examples with practical demonstrations.

**Context Managers:** Provide a clean way to manage resources (files, connections, locks) by automatically handling setup and cleanup operations.

**Key Use Cases:**
- *File operations:* automatic closing of files
- *Database connections:* ensure connections are properly closed
- *Locks and thread synchronization:* automatic acquire/release
- *Temporary resources:* create and cleanup temp files/directories
- *Transaction management:* automatic commit/rollback
- *Exception suppression:* cleanly ignore specific exceptions
- *Timing code execution:* measure performance
- *Configuration overrides:* temporarily change settings

**Syntax:** with context_manager as variable:
    # Code block that uses the resource
    # Cleanup happens automatically when block exits

print("Context Managers: Simplify resource management with automatic cleanup")

## Basic Context Manager

In [ ]:
# Context managers handle setup and teardown automatically using 'with' statement

# Without context manager (manual cleanup - not recommended)
file = open("temp.txt", "w")
file.write("Hello, World!")
file.close()  # Must remember to close

File written successfully


## With Context Manager

In [ ]:
# With context manager (automatic cleanup - recommended)
with open("temp.txt", "w") as file:
    file.write("Hello with context manager!")
# File is automatically closed here, even if an exception occurs

print("File written successfully")

## Custom Context Manager (Class-based)

In [2]:
# Create custom context manager using __enter__ and __exit__ methods

class FileManager:
    """Custom file manager context manager"""
    
    def __init__(self, filename, mode):
        self.filename = filename
        self.mode = mode
        self.file = None
    
    def __enter__(self):
        """Called when entering 'with' block"""
        print(f"Opening {self.filename}")
        self.file = open(self.filename, self.mode)
        return self.file  # This value is assigned to 'as' variable
    
    def __exit__(self, exc_type, exc_value, traceback):
        """Called when exiting 'with' block (even on exception)"""
        print(f"Closing {self.filename}")
        if self.file:
            self.file.close()
        # Return False to propagate exceptions, True to suppress
        return False

# Use custom context manager
with FileManager("test.txt", "w") as f:
    f.write("Custom context manager!")

Opening test.txt
Closing test.txt


## Context Manager with contextlib

In [ ]:
# Create context manager using @contextmanager decorator (simpler approach)

from contextlib import contextmanager

@contextmanager
def open_file(filename, mode):
    """Context manager using decorator"""
    print(f"Opening {filename}")
    file = open(filename, mode)
    try:
        yield file  # Code runs here with the file open
    finally:
        print(f"Closing {filename}")
        file.close()  # Always executes, even on exception

# Use the decorated context manager
with open_file("test2.txt", "w") as f:
    f.write("Using contextlib decorator!")

## Multiple Context Managers

In [3]:
# Use multiple context managers in a single 'with' statement

with open("source.txt", "w") as source, open("dest.txt", "w") as dest:
    source.write("Source content")
    dest.write("Destination content")

print("Both files written and closed automatically")

Both files written and closed automatically


## Database Context Manager

In [4]:
# Simulate database connection management with context manager

class DatabaseConnection:
    """Context manager for database connections"""
    
    def __init__(self, db_name):
        self.db_name = db_name
        self.connection = None
    
    def __enter__(self):
        """Establish database connection"""
        print(f"Connecting to {self.db_name}...")
        self.connection = f"Connection to {self.db_name}"
        return self.connection
    
    def __exit__(self, exc_type, exc_value, traceback):
        """Close database connection"""
        print(f"Closing connection to {self.db_name}")
        self.connection = None
        return False

# Use database context manager
with DatabaseConnection("my_database") as conn:
    print(f"Using: {conn}")
    print("Executing queries...")
# Connection automatically closed

Connecting to my_database...
Using: Connection to my_database
Executing queries...
Closing connection to my_database


## Context Manager with Exception Handling

In [ ]:
# Context managers can handle exceptions gracefully

class SafeOperation:
    """Context manager that suppresses exceptions"""
    
    def __enter__(self):
        print("Starting safe operation")
        return self
    
    def __exit__(self, exc_type, exc_value, traceback):
        if exc_type is not None:
            print(f"Exception occurred: {exc_type.__name__}: {exc_value}")
            return True  # Suppress the exception
        print("Operation completed successfully")
        return False

# Test with no exception
with SafeOperation():
    print("Doing some work...")

# Test with exception (uncomment to see suppression)
# with SafeOperation():
#     raise ValueError("Something went wrong")

print("Continuing after context manager")

## Timer Context Manager

In [ ]:
# Measure execution time using context manager

import time
from contextlib import contextmanager

@contextmanager
def timer(name):
    """Context manager to measure execution time"""
    start = time.time()
    print(f"[{name}] Starting...")
    yield  # Code block executes here
    end = time.time()
    print(f"[{name}] Completed in {end - start:.4f} seconds")

# Measure task execution time
with timer("Task 1"):
    time.sleep(0.5)
    print("  Processing...")

## Temporary Directory Context Manager

In [ ]:
# Automatically create and cleanup temporary directories

import os
import tempfile
from contextlib import contextmanager

@contextmanager
def temporary_directory():
    """Create temporary directory and clean it up afterwards"""
    temp_dir = tempfile.mkdtemp()
    print(f"Created temporary directory: {temp_dir}")
    try:
        yield temp_dir  # Work with temp directory
    finally:
        import shutil
        shutil.rmtree(temp_dir)
        print(f"Removed temporary directory: {temp_dir}")

# Use temporary directory
with temporary_directory() as temp_dir:
    # Create a file in temp directory
    temp_file = os.path.join(temp_dir, "temp.txt")
    with open(temp_file, "w") as f:
        f.write("Temporary data")
    print(f"  Working in: {temp_dir}")
# Directory automatically deleted

## Suppress Context Manager

In [ ]:
# Suppress specific exceptions using contextlib.suppress

from contextlib import suppress

# Traditional exception handling
try:
    int("abc")
except ValueError:
    print("Caught ValueError (traditional way)")

# Using suppress (cleaner for ignoring exceptions)
with suppress(ValueError):
    int("xyz")  # Exception is silently ignored

print("ValueError suppressed - code continues")

## Redirect stdout Context Manager

In [ ]:
# Capture or redirect stdout using context manager

from contextlib import redirect_stdout
import io

print("Normal output")

# Capture print output to StringIO
f = io.StringIO()
with redirect_stdout(f):
    print("This goes to StringIO")
    print("Not printed to console")

# Get captured output
output = f.getvalue()
print(f"Captured output: {output.strip()}")

## Lock Context Manager

In [ ]:
# Use locks with context managers for thread safety

import threading

class SharedResource:
    """Thread-safe resource using lock context manager"""
    
    def __init__(self):
        self.lock = threading.Lock()
        self.value = 0
    
    def increment(self):
        with self.lock:  # Automatically acquire and release lock
            print(f"Incrementing from {self.value}")
            self.value += 1
            return self.value

# Use shared resource
resource = SharedResource()
print(f"Final value: {resource.increment()}")

## Practical Example: Configuration Manager

In [ ]:
# Temporarily override configuration settings

from contextlib import contextmanager

@contextmanager
def config_override(key, value):
    """Temporarily override a configuration value"""
    config = {"debug": False, "log_level": "INFO"}
    
    old_value = config.get(key)
    config[key] = value
    print(f"Config: {key} = {value}")
    
    try:
        yield config
    finally:
        config[key] = old_value
        print(f"Config restored: {key} = {old_value}")

# Temporarily enable debug mode
with config_override("debug", True) as cfg:
    print(f"  Running with debug={cfg['debug']}")
# Debug mode automatically restored

## Practical Example: Transaction Manager

In [ ]:
# Manage database-like transactions with automatic commit/rollback

class Transaction:
    """Context manager for transaction management"""
    
    def __init__(self, name):
        self.name = name
        self.committed = False
    
    def __enter__(self):
        print(f"[{self.name}] Transaction started")
        return self
    
    def __exit__(self, exc_type, exc_value, traceback):
        if exc_type is None:
            self.commit()
        else:
            self.rollback()
            return False  # Re-raise exception
    
    def commit(self):
        print(f"[{self.name}] Transaction committed")
        self.committed = True
    
    def rollback(self):
        print(f"[{self.name}] Transaction rolled back")
        self.committed = False

# Successful transaction
with Transaction("Payment") as txn:
    print("  Processing payment...")
    print("  Payment successful")

# Failed transaction (exception triggers rollback)
try:
    with Transaction("Refund") as txn:
        print("  Processing refund...")
        raise ValueError("Insufficient funds")
except ValueError:
    print("  Transaction failed and rolled back")

## Cleanup

In [5]:
# Clean up temporary files created in examples

import os

for f in ["temp.txt", "test.txt", "test2.txt", "source.txt", "dest.txt"]:
    if os.path.exists(f):
        os.remove(f)
        print(f"Removed: {f}")

print("\nCleanup complete!")

Removed: temp.txt
Removed: test.txt
Removed: source.txt
Removed: dest.txt

Cleanup complete!
